In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
start_url = "https://trak.in/india-startup-funding-investment-2015/"
html_response = requests.get(start_url)
html_response.status_code

200

In [3]:
soup = BeautifulSoup(html_response.content, 'html.parser')

In [4]:
more_data_urls = [start_url]

for h3_tag in soup.find_all(name="h3"):
    more_data_urls.append(h3_tag.find(name='a').get('href'))

more_data_urls[:5]

['https://trak.in/india-startup-funding-investment-2015/',
 'https://trak.in/indian-startup-funding-and-investment-chart-january-2018/',
 'https://trak.in/indian-startup-funding-and-investment-chart-february-2018/',
 'https://trak.in/indian-startup-funding-and-investment-chart-march-2018/',
 'https://trak.in/indian-startup-funding-investment-chart-april-2018/']

In [5]:
import pandas as pd

In [6]:
new_row_list = []
column_name = ['Sr. No.', 'Date (dd/mm/yyyy)', 'Startup Name', 'Industry/ Vertical', 'Sub-Vertical', 'City / Location', 'Investors’ Name', 'Investment Type', 'Amount (in USD)']
more_data_urls = set(more_data_urls)

urls_count = 1
for url in more_data_urls:
    html_response = requests.get(url)
    html_response.status_code
    soup = BeautifulSoup(html_response.content, 'html.parser')
    
    class_list = []
    for element in soup.find_all(class_=True):
        class_list.extend(element["class"])
    class_list = [cls for cls in class_list if 'tablepress-id-' in cls] 
    
    if len(class_list) < 1:
        skip_first_row = True
        class_list.append(None)
        for class_ in class_list:
            tbl=soup.find(name='table') #, class_=class_)

            n_rows = 0
            for tr in tbl.find_all('tr'):
                if skip_first_row == True:
                    skip_first_row = False
                    continue
                new_row = {}
                for col_id, td in enumerate(tr.find_all('td')):
                    if col_id < len(column_name):
                        new_row[column_name[col_id]] = td.text
                if not new_row == {}:
                    n_rows += 1
                    new_row_list.append(new_row)
            #print("class_list-old:", class_, len(new_row_list), n_rows, url)
    else:
        for class_ in class_list:
            tbl=soup.find(name='table', class_=class_)

            n_rows = 0
            for tr in tbl.find_all('tr'):
                new_row = {}
                for col_id, td in enumerate(tr.find_all('td')):
                    if col_id < len(column_name):
                        new_row[column_name[col_id]] = td.text
                if not new_row == {}:
                    n_rows += 1
                    new_row_list.append(new_row)
            #print("class_list-new :", class_, len(new_row_list), n_rows, url)

data = pd.DataFrame(new_row_list, columns=column_name)
print("Data shape :", data.shape)

Data shape : (3511, 9)


In [7]:
data.head()

,Sr. No.,Date (dd/mm/yyyy),Startup Name,Industry/ Vertical,Sub-Vertical,City / Location,Investors’ Name,Investment Type,Amount (in USD)
0,1,01/08/2015,Snapdeal,Ecommerce Marketplace,New Delhi,"Alibaba, Foxconn, Softbank",Private Equity,"500,000,000",Late Stage Round 7th (More details)
1,2,01/08/2015,Ressy,Restaurant Discount app,Pune,Mohandas Pai & others,Seed Funding,"400,000",
2,3,03/08/2015,Cuelearn,Tech-enabled learning,New Delhi,Alok Mittal,Seed Funding,N/A,
3,4,03/08/2015,Connect India,ECommerce Logistics provider,New Delhi,Aavishkaar,Private Equity,"5,000,000",Series A
4,5,03/08/2015,The Home Salon,on-demand home beauty and wellness portal,Mumbai,VentureNursery,Seed Funding,N/A,


In [8]:
data.to_csv("Startup funding.csv")